In [2]:
!pip install pyodbc 
# ODBC的DRIVER(驅動程式)

Active code page: 936


### ----開始執行----
#### 負責人 09156248 黃登祥

In [3]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
%autosave 120

【現在時間】2022/12/19 16:48:33
【所在目錄】C:\Users\User\Desktop\資料工程
【Python】3.9.13
【pymssql】2.2.7
【pyodbc】4.0.35


Autosaving every 120 seconds


In [4]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

### ----更改股票代號----

In [33]:
stock_id = "6446" #1605 1218 6446
url = "https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID=" + stock_id
#payloads = {"STOCK_ID":stock_id}
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [34]:
url = "https://goodinfo.tw/tw/ShowK_ChartFlow.asp?RPT_CAT=DR_3M&STOCK_ID="+stock_id+"&CHT_CAT=WEEK"
#,data=payloads
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

flow_table = pd.read_html(resp.text)

In [35]:
url = "https://goodinfo.tw/tw/ShowK_ChartFlow.asp?RPT_CAT=DR_1Y&STOCK_ID="+stock_id+"&CHT_CAT=WEEK"
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

flow_table2 = pd.read_html(resp.text)

In [36]:
df = flow_table[17].dropna(axis='rows')
#df.drop("季均線乖離率換算價格",axis=1)
df.columns = df.columns.droplevel()
df = df.drop(['-40%','-30%','-20%','-10%','40%','30%','20%','10%'],axis=1)
df = df.rename(columns = {"均線":"季均線"})
print(type(df))
df.head()


<class 'pandas.core.frame.DataFrame'>


,交易週別,收盤價,漲跌價,漲跌幅,目前乖離(%),季均線
0,22W51,525,-12,-2.23%,+3.84,505.6
1,22W50,537,+5,+0.94%,+5.16,510.7
2,22W49,532,+26,+5.14%,+3.9,512
3,22W48,506,-22,-4.17%,-1.53,513.9
4,22W47,528,+20,+3.94%,+1.44,520.5


In [37]:
df2 = flow_table2[17].dropna(axis='rows')
df2.columns = df2.columns.droplevel()
df = df.assign(年均線 = df2['均線']).dropna(axis='rows')
df = df.drop(df[df["交易週別"]=="交易週別"].index)
df.reset_index(inplace=False,drop=False)
df

,交易週別,收盤價,漲跌價,漲跌幅,目前乖離(%),季均線,年均線
0,22W51,525,-12,-2.23%,+3.84,505.6,424.3
1,22W50,537,+5,+0.94%,+5.16,510.7,421.1
2,22W49,532,+26,+5.14%,+3.9,512,415.5
3,22W48,506,-22,-4.17%,-1.53,513.9,409
4,22W47,528,+20,+3.94%,+1.44,520.5,402.6
...,...,...,...,...,...,...,...
165,20W04,114.5,-1.5,-1.29%,+4.28,109.8,128.7
166,20W03,116,+1,+0.87%,+5.65,109.8,129.8
167,20W02,115,+2.5,+2.22%,+4.85,109.7,131
168,20W01,112.5,+11.5,+11.4%,+3.04,109.2,132.1


In [38]:
products_list = df.values.tolist()
print(products_list[138])
del products_list[139]
del products_list[138]
del products_list[119]
del products_list[118]
del products_list[99]
del products_list[98]
del products_list[79]
del products_list[78]
del products_list[59]
del products_list[58]
del products_list[39]
del products_list[38]
del products_list[19]
del products_list[18]
lists_num=len(products_list)
print(len(products_list))
print(products_list)


['20W15', '77', '+6.3', '+8.91%', '-17.7', '93.51', '112.3']
140
[['22W51', '525', '-12', '-2.23%', '+3.84', '505.6', '424.3'], ['22W50', '537', '+5', '+0.94%', '+5.16', '510.7', '421.1'], ['22W49', '532', '+26', '+5.14%', '+3.9', '512', '415.5'], ['22W48', '506', '-22', '-4.17%', '-1.53', '513.9', '409'], ['22W47', '528', '+20', '+3.94%', '+1.44', '520.5', '402.6'], ['22W46', '508', '+5', '+0.99%', '-3.16', '524.6', '394.5'], ['22W45', '503', '+55', '+12.3%', '-4.52', '526.8', '386.5'], ['22W44', '448', '0', '0%', '-16.4', '535.7', '378.7'], ['22W43', '448', '-38.5', '-7.91%', '-17.8', '545', '372'], ['22W42', '486.5', '-1', '-0.21%', '-11.6', '550.5', '365.1'], ['22W41', '487.5', '+4', '+0.83%', '-11.2', '549.1', '357.1'], ['22W40', '483.5', '-96.5', '-16.6%', '-11.8', '548.4', '349.2'], ['22W39', '580', '-11', '-1.86%', '+5.54', '549.6', '341.8'], ['22W38', '591', '+36', '+6.49%', '+8.78', '543.3', '332.5'], ['22W37', '555', '-1', '-0.18%', '+3.83', '534.5', '323'], ['22W36', '556',

In [12]:
connsqlodbc= pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=124.218.194.84;"
                      "DATABASE=testDB;UID=testuser;PWD=323")


### 創表

In [24]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令


sqlStr ='''create table 股票各月乖離與均線表
(
	股票各月乖離與均線表編號 nvarchar(20) primary key
	,收盤價 nvarchar(20) NULL
	,漲跌價 nvarchar(20) Null
	,[目前乖離(%)] nvarchar(20) Null
	,季均線 nvarchar(20) Null
    ,年均線 nvarchar(20) Null
	,交易月份 nvarchar(5) not Null
	,股票代號 nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

### 資料入庫

In [39]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# R+代號+發月別
# MYSQL指令
for i in range(0,lists_num):
    Rstock_id='R'+str(stock_id)+str(products_list[i][0])
    sqlStr = f""" 
    Insert into 股票各月乖離與均線表 ([股票各月乖離與均線表編號], 
    [收盤價], [漲跌價],
    [目前乖離(%)],[季均線],[年均線],
    交易月份,股票代號)
    values ('{str(Rstock_id)}',
    '{products_list[i][1]}','{products_list[i][2]}','{products_list[i][4]}','{products_list[i][5]}','{products_list[i][6]}',
    '{products_list[i][0]}','{stock_id}')
    """
    cursor.execute(sqlStr)
'''交易週別	收盤價	漲跌價	漲跌幅	目前乖離(%)	季均線	年均線'''
print(sqlStr)

 
    Insert into 股票各月乖離與均線表 ([股票各月乖離與均線表編號], 
    [收盤價], [漲跌價],
    [目前乖離(%)],[季均線],[年均線],
    交易月份,股票代號)
    values ('R644619W52',
    '101','+0.5','-7.4','109.1','133.3',
    '19W52','6446')
    


In [ ]:
'''
R+代號+發月別
交易月份	收盤價	漲跌價		目前乖離(%)	均線'''

In [5]:
stock_id = "6446"
url = "https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID=" + stock_id
#payloads = {"STOCK_ID":stock_id}
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [6]:
url = "https://goodinfo.tw/tw/ShowK_ChartFlow.asp?RPT_CAT=DR_3M&STOCK_ID="+stock_id+"&CHT_CAT=WEEK"
#,data=payloads
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

flow_table = pd.read_html(resp.text)
url = "https://goodinfo.tw/tw/ShowK_ChartFlow.asp?RPT_CAT=DR_1Y&STOCK_ID="+stock_id+"&CHT_CAT=WEEK"
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

flow_table2 = pd.read_html(resp.text)

In [7]:
#加入周K
url = "https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID="+stock_id+"&CHT_CAT2=WEEK&PRICE_ADJ=F"
#,data=payloads
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

K_table = pd.read_html(resp.text)

In [8]:
df = flow_table[17].dropna(axis='rows')
#df.drop("季均線乖離率換算價格",axis=1)
df.columns = df.columns.droplevel()
df = df.drop(['-40%','-30%','-20%','-10%','40%','30%','20%','10%'],axis=1)
df = df.rename(columns = {"均線":"季均線"})
df

,交易週別,收盤價,漲跌價,漲跌幅,目前乖離(%),季均線
0,22W52,523,-2,-0.38%,+4.35,501.2
1,22W51,525,-12,-2.23%,+3.84,505.6
2,22W50,537,+5,+0.94%,+5.16,510.7
3,22W49,532,+26,+5.14%,+3.9,512
4,22W48,506,-22,-4.17%,-1.53,513.9
...,...,...,...,...,...,...
166,20W04,114.5,-1.5,-1.29%,+4.28,109.8
167,20W03,116,+1,+0.87%,+5.65,109.8
168,20W02,115,+2.5,+2.22%,+4.85,109.7
169,20W01,112.5,+11.5,+11.4%,+3.04,109.2


In [9]:
df2 = flow_table2[17].dropna(axis='rows')
df2.columns = df2.columns.droplevel()
df = df.assign(年均線 = df2['均線']).dropna(axis='rows')
df = df.drop(df[df["交易週別"]=="交易週別"].index)
df.reset_index(inplace=False,drop=False)
df

,交易週別,收盤價,漲跌價,漲跌幅,目前乖離(%),季均線,年均線
0,22W52,523,-2,-0.38%,+4.35,501.2,427.9
1,22W51,525,-12,-2.23%,+3.84,505.6,424.3
2,22W50,537,+5,+0.94%,+5.16,510.7,421.1
3,22W49,532,+26,+5.14%,+3.9,512,415.5
4,22W48,506,-22,-4.17%,-1.53,513.9,409
...,...,...,...,...,...,...,...
166,20W04,114.5,-1.5,-1.29%,+4.28,109.8,128.7
167,20W03,116,+1,+0.87%,+5.65,109.8,129.8
168,20W02,115,+2.5,+2.22%,+4.85,109.7,131
169,20W01,112.5,+11.5,+11.4%,+3.04,109.2,132.1


In [10]:
#交易週別
dfk = K_table[25].dropna(axis='columns')
dfk.columns = dfk.columns.droplevel()
dfk = dfk.drop(dfk[dfk["交易週別"]=="交易週別"].index)
dfk.reset_index(inplace=False,drop=False)

df = df.assign(開盤 = dfk['開盤']).dropna(axis='rows')
df = df.assign(最高 = dfk['最高']).dropna(axis='rows')
df = df.assign(最低 = dfk['最低']).dropna(axis='rows')

df = df.rename(columns = {"開盤":"週開盤"})
df = df.rename(columns = {"收盤價":"週收盤"})
df = df.rename(columns = {"最高":"週最高"})
df = df.rename(columns = {"最低":"週最低"})

dfk.reset_index(inplace=False,drop=False)

df

,交易週別,週收盤,漲跌價,漲跌幅,目前乖離(%),季均線,年均線,週開盤,週最高,週最低
0,22W52,523,-2,-0.38%,+4.35,501.2,427.9,523,530,522
1,22W51,525,-12,-2.23%,+3.84,505.6,424.3,536,540,518
2,22W50,537,+5,+0.94%,+5.16,510.7,421.1,530,552,516
3,22W49,532,+26,+5.14%,+3.9,512,415.5,500,535,488.5
4,22W48,506,-22,-4.17%,-1.53,513.9,409,530,540,503
...,...,...,...,...,...,...,...,...,...,...
166,20W04,114.5,-1.5,-1.29%,+4.28,109.8,128.7,114.5,116,113.5
167,20W03,116,+1,+0.87%,+5.65,109.8,129.8,116,117.5,112.5
168,20W02,115,+2.5,+2.22%,+4.85,109.7,131,114.5,124,113.5
169,20W01,112.5,+11.5,+11.4%,+3.04,109.2,132.1,101.5,119.5,99.9


In [11]:

products_list = df.values.tolist()

lists_num=len(products_list)
print(len(products_list))
print(products_list)

155
[['22W52', '523', '-2', '-0.38%', '+4.35', '501.2', '427.9', '523', '530', '522'], ['22W51', '525', '-12', '-2.23%', '+3.84', '505.6', '424.3', '536', '540', '518'], ['22W50', '537', '+5', '+0.94%', '+5.16', '510.7', '421.1', '530', '552', '516'], ['22W49', '532', '+26', '+5.14%', '+3.9', '512', '415.5', '500', '535', '488.5'], ['22W48', '506', '-22', '-4.17%', '-1.53', '513.9', '409', '530', '540', '503'], ['22W47', '528', '+20', '+3.94%', '+1.44', '520.5', '402.6', '511', '529', '480'], ['22W46', '508', '+5', '+0.99%', '-3.16', '524.6', '394.5', '499', '522', '477.5'], ['22W45', '503', '+55', '+12.3%', '-4.52', '526.8', '386.5', '454', '517', '445.5'], ['22W44', '448', '0', '0%', '-16.4', '535.7', '378.7', '452', '460', '410'], ['22W43', '448', '-38.5', '-7.91%', '-17.8', '545', '372', '478', '489', '441.5'], ['22W42', '486.5', '-1', '-0.21%', '-11.6', '550.5', '365.1', '480', '495', '431.5'], ['22W41', '487.5', '+4', '+0.83%', '-11.2', '549.1', '357.1', '477', '507', '465'], ['2

In [13]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令


sqlStr ='''create table 股票各月乖離與均線表
(
	股票各週乖離與均線表編號 nvarchar(20) primary key
	,週開盤 nvarchar(20) Null
    ,週收盤 nvarchar(20) NULL
    ,週最高 nvarchar(20) Null
    ,週最低 nvarchar(20) Null
	,漲跌價 nvarchar(20) Null
	,[目前乖離(%)] nvarchar(20) Null
	,季均線 nvarchar(20) Null
    ,年均線 nvarchar(20) Null
	,交易週別 nvarchar(5) not Null
	,股票代號 nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

In [15]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# R+代號+發月別
# MYSQL指令
for i in range(0,lists_num):
    Rstock_id='R'+str(stock_id)+str(products_list[i][0])
    sqlStr = f""" 
    Insert into 股票各月乖離與均線表 (
    [股票各週乖離與均線表編號], 
    [週開盤],[週收盤], [週最高],
    [週最低],[漲跌價],[目前乖離(%)]
    ,[季均線],[年均線],
    交易週別,股票代號)
    values ('{str(Rstock_id)}',
    '{products_list[i][7]}','{products_list[i][1]}','{products_list[i][8]}','{products_list[i][9]}',
    '{products_list[i][2]}','{products_list[i][4]}','{products_list[i][5]}','{products_list[i][6]}',
    '{products_list[i][0]}','{stock_id}')
    """
    cursor.execute(sqlStr)

print(sqlStr)
"    0       1    2     3       4           5     6      7     8     9    "
"交易週別	週收盤	漲跌價	漲跌幅	目前乖離(%)	季均線	年均線	週開盤	週最高	週最低"

 
    Insert into 股票各月乖離與均線表 (
    [股票各週乖離與均線表編號], 
    [週開盤],[週收盤], [週最高],
    [週最低],[漲跌價],[目前乖離(%)]
    ,[季均線],[年均線],
    交易週別,股票代號)
    values ('R644619W52',
    '100.5','101','101.5','98.1',
    '+0.5','-7.4','109.1','133.3',
    '19W52','6446')
    


'交易週別\t週收盤\t漲跌價\t漲跌幅\t目前乖離(%)\t季均線\t年均線\t週開盤\t週最高\t週最低'